In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 31: Likelihood Ratio Tests

Recall maximum likelihood estimators. These are obtained by maximizing the likelihood function with respect to $\theta$, the parameter of interest. Let's go through an example:

### Example 1: Poisson Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the Poisson distribution with unknown parameter $\lambda$. Find $\hat{\lambda}_{ML}$, the maximum likelihood estimate of $\lambda$. 

$
\begin{align}
L(\lambda\mid\textbf{x})&=\prod_{i=1}^n f(x_i;\lambda)\\
&=\prod_{i=1}^n e^{-\lambda}\lambda^{x_i}\frac{1}{x_i!}\\
L(\lambda\mid\textbf{x})&=e^{-n\lambda}\lambda^{\left(\sum_{i=1}^n x_i\right)} \frac{1}{\prod_{i=1}^n x_i}\\
l(\lambda\mid\textbf{x})&=-n\lambda\log(e) + \left(\sum_{i=1}^n x_i\right)\log(\lambda) - \sum_{i=1}^n x_i!\\
l(\lambda\mid\textbf{x})&=-n\lambda + n\bar{X}\log{\lambda} - \sum_{i=1}^n x_i!\\
\frac{d}{d\lambda}l(\lambda\mid\textbf{x})&=-n + \frac{n\bar{X}}{\lambda} - 0\\
0&=n\left(-1 + \frac{\bar{X}}{\hat{\lambda}_{ML}}\right)\\
\hat{\lambda}_{ML}&=\bar{X}
\end{align}
$

### Likelihood Ratio Tests

Assume you are testing a hypothesis:
$$
H_0: \theta=\theta_0
$$
$$
H_1: \theta\neq \theta_0
$$

The idea behind a likelihood ratio test is to compare the likelihood of the hypothesized value ($L(\theta_0\mid\textbf{x})$) to the maximum likelihood given the data ($L(\hat{\theta}_{ML}\mid\textbf{x})$). If the hypothesized value of $\theta$ were feasible, the likelihood under $\theta_0$ should be close to the max. If the hypothesized value of $\theta$ were not feasible, $L(\theta_0\mid\textbf{x})$ should be much smaller. To make the comparison, we consider the likelihood ratio test statistic, $\Lambda$: 

$$
\Lambda=\frac{L(\theta_0\mid\textbf{x})}{L(\hat{\theta}_{ML}\mid\textbf{x})}
$$

Because $\hat{\theta}_{ML}$ is the maximum likelihood estimator, this ratio has a maximum value of 1. Large values of $\Lambda$ (close to 1) indicate that $\theta_0$ is feasible (lack of evidence to reject). Small values of $\Lambda$ (close to 0) indicate $\theta_0$ is not feasible (evidence to reject). 

But how close to 0 is "close"? 

To evaluate this, we will take advantage of a helpful result. It turns out that if the null hypothesis were true, $-2\log \Lambda$ approximately follows the chi-squared distribution with 1 degree of freedom. The proof is outside the scope of this class. 

[We have not yet talked about the chi-squared distribution. To learn more, consult scipy help (`scipy.stats.chi2`). This distribution has one parameter that we care about: degrees of freedom, referenced in scipy as `df`. Bottom line, a random variable that has a chi-squared distribution with `df` degrees of freedom has a domain of $[0,\infty)$ and an expected value of `df`.]

$$
-2\log \Lambda = 2\left[l(\hat{\theta}_{ML}\mid \textbf{x})-l(\theta_0\mid \textbf{x})\right] \approx \textsf{Chisq}(1)
$$

It is a little bit more intuitive to consider $-2\log \Lambda$; large values of this test statistic indicate evidence to reject the null, which is consistent with most other hypothesis tests we've looked at. 

### Example 2: Likelihood ratio test on $\lambda$ from Poisson distribution

(Example taken from Pruim 2011). An instructor believes the the number of students who arrive late for class follows a Poisson process with an average of 1 late arrival per lesson. In 10 consecutive lessons, he found the following number of late arrivals: (1,1,0,4,2,1,3,0,0,2). Conduct a likelihood ratio test on the following hypothesis:

$$
H_0: \lambda = 1
$$
$$
H_1: \lambda \neq 1
$$

$
\begin{align}
\Lambda&=\frac{L(\lambda=1\mid\textbf{x})}{L(\hat{\lambda}_{ML}\mid\textbf{x})}\\
&=\frac{e^{-n(1)} (1)^{\sum_{i=1}^n x_i} \frac{1}{\prod_{i=1}^n x_i!} }{e^{-n(\bar{X})} (\bar{X})^{\sum_{i=1}^n x_i} \frac{1}{\prod_{i=1}^n x_i!} }\\
&=\frac{e^{-n}}{e^{-n\bar{X}} \bar{X}^{\sum_{i=1}^n x_i}}\\
&=\frac{e^{\sum_{i=1}^n x_i}}{e^n \bar{X}^{\sum_{i=1}^n x_i}}\\
n&=10\quad\bar{X}=1.4\quad\sum_{i=1}^n x_i=14\\
\Lambda&=\frac{e^{14}}{e^{10} 1.4^{14}}\\
\Lambda&=0.491
\end{align}
$

$
\begin{align}
-2\log\Lambda&=-2\log(0.491)\\
&=1.421
\end{align}
$

In [5]:
p=1-stats.chi2.cdf(x=1.421,df=1)
print("The p-value for the hypothesis H0 is " + str(p))

The p-value for the hypothesis H0 is 0.2332388781087974


*Since the likelihood ratio of $H_0$ is small, with a p-value much greater than 0.05, we find no evidence to reject the null hypothesis.*

#### Alternative Method

In this method, I simply use the sample mean as the test statistic. The observed value was 1.4. In the below, I simulate under the hypothesized $\lambda$, 1, and determine how often the sample mean was further away from 1 as was 1.4. 

In [7]:
x = [1,1,0,4,2,1,3,0,0,2]
ts2=[np.mean(stats.poisson.rvs(1,size=10)) for _ in np.arange(10000)]
np.mean(ts2>=np.mean(x))+np.mean(ts2<=(1-(np.mean(x)-1)))

0.2612